## Using Keras to run the CASP dataset

In this notebook, we show how to use Keras to the run regression test on the CASP dataset. I will introduce 4 different methods and go through each of them by grid search on 4 different combinations of parameters. 

### First, we load the dependencies. 

In [1]:
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
import keras


Using TensorFlow backend.


### Test GPU support

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Load in dataset using Pandas

We use `read_csv` to the read in and also select the X and y matrices. 

In [4]:
data = pd.read_csv('winequality-red.csv', header=0)

In [6]:
# select X and y matrices.
y = data['quality'].as_matrix()
X = data.drop(labels='quality', axis=1).as_matrix()

In [7]:
# check the shape of X and y
y.shape, X.shape

((1599,), (1599, 11))

## Start training and parameter selection

We the use the API from `scikit-learn` to conduct a grid search on parameters. (Here I use 5-fold cross-validation to save training time.) We also need `KerasRegressor` to use sklearn together with Keras

In [ ]:
% time
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

## 5-fold cross-validation 

Here `new2_advanced_hidden_model` is the name for my best model. 

In [13]:
% time

def new2_advanced_hidden_model():
    model = Sequential()
    model.add(Dense(12, activation='relu', input_dim=11))
    model.add(Dense(18, activation='relu', input_dim=12))
    model.add(Dense(13, activation='relu', input_dim=18))
    model.add(Dense(10, activation='relu', input_dim=13))
    model.add(Dense(7, activation='relu', input_dim=10))
    model.add(Dense(1, activation='linear', input_dim=7))
#     model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.7 µs


In [14]:
## Here are the params for grid search
parameters = {
    'epochs' : [800, 700],
    'batch_size' : [150, 140], # The larger the batch size, the faster the model will run.
        'verbose' : [0] 
}


In [15]:
# fit the network
reg_hidden_new2 = GridSearchCV(KerasRegressor(build_fn=new2_advanced_hidden_model), parameters, cv = 5, scoring = 'neg_mean_absolute_error', return_train_score = True)
reg_hidden_new2.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f9146cae390>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'epochs': [800, 700], 'batch_size': [150, 140], 'verbose': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [16]:
# use pandas to check the cv results
pd.DataFrame(reg_hidden_new2.cv_results_)


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_batch_size,param_epochs,param_verbose,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,25.718922,0.068409,-0.523236,-0.475656,150,800,0,"{'batch_size': 150, 'epochs': 800, 'verbose': 0}",2,-0.504227,...,-0.501404,-0.489269,-0.532049,-0.467236,-0.498269,-0.478596,0.691282,0.032084,0.030918,0.007721
1,23.867300,0.139691,-0.527965,-0.474639,150,700,0,"{'batch_size': 150, 'epochs': 700, 'verbose': 0}",3,-0.507181,...,-0.505725,-0.479929,-0.566245,-0.444288,-0.502097,-0.487652,0.623886,0.033596,0.028262,0.015629
2,31.147727,0.214613,-0.518686,-0.480870,140,800,0,"{'batch_size': 140, 'epochs': 800, 'verbose': 0}",1,-0.494653,...,-0.505225,-0.476792,-0.553071,-0.477485,-0.494285,-0.482516,0.729665,0.024810,0.025649,0.005668
3,28.989724,0.284276,-0.528049,-0.487590,140,700,0,"{'batch_size': 140, 'epochs': 700, 'verbose': 0}",4,-0.515618,...,-0.519418,-0.504512,-0.560229,-0.474307,-0.509923,-0.496622,0.660966,0.025649,0.018119,0.011552


## Here are out best scores with best parameters

In [17]:
reg_hidden_new2.best_score_, reg_hidden_new2.best_score_

(-0.5186856677190747, -0.5186856677190747)

In [19]:
reg_hidden_new2.best_params_


{'batch_size': 140, 'epochs': 800, 'verbose': 0}